In [2]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=b7365411b83168a0c6d4bac7c0968ec64663987427ddadf12019f4efd25a4b43
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import networkx as nx
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset
import os

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
text = """Ad sales boost Time Warner profit  Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.  Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. "Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility," chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.  TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake. """

In [5]:
def create_embeddings():
  embeddings = {}
  f = open("/content/drive/MyDrive/Datasets/Embeddings/GloVe/glove.6B.50d.txt")
  f = f.readlines()
  for line in f:
   values = line.strip().split()
   embeddings[values[0]] = np.array(values[1:]).astype('float64')
  return embeddings

In [6]:
embeddings = create_embeddings()

In [ ]:
embeddings

In [31]:
def cleanTokens(tokens):
  lemmatizer = WordNetLemmatizer()

  def removeStopWords(token):
    if token not in set(stopwords.words('english')):
      return lemmatizer.lemmatize(token)

  removeStopWords = np.vectorize(removeStopWords)
  cleaned_tokens = removeStopWords(tokens)
  cleaned_tokens = np.delete(cleaned_tokens, cleaned_tokens=='None')
  return cleaned_tokens

def cleanText(document):
  text = document.lower()
  text = re.sub('[^a-zA-Z]',' ',text)
  tokens = word_tokenize(text)
  cleaned_tokens = cleanTokens(tokens)
  return cleaned_tokens

def createCooccurrences(cleaned_tokens):
  finder = BigramCollocationFinder.from_words(cleaned_tokens)
  co_occurrences = finder.ngram_fd.items()
  return co_occurrences

cleaned_tokens = cleanText(text)
co_occurrences = createCooccurrences(cleaned_tokens)
def createPyG(co_occurrences, embeddings=embeddings, cleaned_tokens=cleaned_tokens):
  co_occurrences = list(co_occurrences)
  word_to_idx = {word: idx for idx, word in enumerate(cleaned_tokens)}
  def extractGraphData(co_occurrence, word_to_idx = word_to_idx):
    (word1, word2), weight = co_occurrence
    return word_to_idx[word1], word_to_idx[word2], weight

  def word2vec(word, embeddngs = embeddings):
    return embeddings[word]
  extractGraphData = np.vectorize(extractGraphData)
  word2vec = np.vectorize(word2vec)
  graph_data = extractGraphData(co_occurrences)
  weights = graph_data[:,2]
  nodes = graph_data[:,0:2]
  node_index = word2vec(cleaned_tokens)

  return weights, nodes, node_index

In [32]:
createPyG(co_occurrences)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2360: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  args = [asarray(arg) for arg in args]


KeyError: ignored